# Transporter analysis of bacillus mother-spore

In [1]:
from __future__ import print_function, division, absolute_import

import sys

import qminospy
from qminospy.me2 import ME_NLP

# python imports
from copy import copy
import re
from os.path import join, dirname, abspath
import sys
from collections import defaultdict
import pickle

# third party imports
import pandas as pd
import cobra
from tqdm import tqdm
import numpy as np
import scipy

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import bacillusme
from bacillusme import (transcription, translation, flat_files, generics, formulas, compartments)
from cobrame.util.helper_functions import *

import copy
from scipy import stats
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
ecoli_files = dirname(abspath(bacillusme.__file__))
pd.set_option('display.max_colwidth', None)


/home/jt/work/cobrapy-0.5.11/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/jt/work/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/work/cobrapy-0.5.11/cobra/__init__.py
/home/jt/work/cobrame/cobrame/__init__.py
/home/jt/work/sporeme/bacillusme/__init__.py


# Amino acids

In [2]:
with open("./sporeme_solution_v4_proteomics_spoAQ.pickle", "rb") as outfile:
    sporeme = pickle.load(outfile)

In [3]:
pd.DataFrame.from_dict({'flux':sporeme.solution.x_dict}).to_csv('Supplementary File 2.csv')

In [4]:
# for r in get_transport_reactions(sporeme,'met__L_s',comps=['c','s']):
#     print(r.id)
#     r.bounds = (0,0)
# solve_me_model(sporeme, max_mu = 0.1, min_mu = .01, using_soplex=False, precision = 1e-3,growth_key = 'sigma')

In [5]:
if sporeme.solution: print(sporeme.solution.x_dict['biomass_dilution_s'])

0.017942047119140624


In [7]:
dct = {}
for m in sporeme.metabolites.query(re.compile('^[a-z]{3}__L_s|^gly_s$')):
    df = flux_based_reactions(sporeme,m.id,only_types=['MetabolicReaction'])
    prod = df[df['met_flux']>0]['met_flux']
    prod = prod.div(prod.sum())
    top = df.sort_values('met_flux').iloc[[-1]]
    print(m)
    dct[m.id] = {}
    if top['met_flux'][0] > 0:
        print(top['reaction'][0])
        dct[m.id]["reaction"] = top['reaction'][0]
        print(top['met_flux'][0])
        dct[m.id]["met_flux"] = top['met_flux'][0]
        print('{} %'.format(str(prod.loc[top.index[0]]*100)))
        dct[m.id]["percentage"] = '{} %'.format(str(prod.loc[top.index[0]]*100))
    else:
        print('No production')
    print('\n')

met__L_s
atp_s + h2o_s + met__L_c --> -4.27350427350427e-6*sigma PORE-CPLX_s + adp_s + h_s + met__L_s + pi_s
0.00048509737247183564
100.0 %


trp__L_s
h_c + trp__L_c --> -4.27350427350427e-6*sigma PORE-CPLX_s + h_s + trp__L_s
0.00015947163451364514
100.0 %


glu__L_s
akg_s + h_s + nadh_s + nh4_s --> -4.27350427350427e-6*sigma BSU22960-MONOMER_s + glu__L_s + h2o_s + nad_s
0.00810987214544222
83.93457550816719 %


ser__L_s
h_c + ser__L_c --> -4.27350427350427e-6*sigma PORE-CPLX_s + h_s + ser__L_s
0.0032057865794111124
100.0 %


lys__L_s
h_s + lys__L_c --> -4.27350427350427e-6*sigma PORE-CPLX_s + h_c + lys__L_s
0.0018966915533663322
100.0 %


phe__L_s
glu__L_s + phpyr_s --> -4.27350427350427e-6*sigma BSU09570-MONOMER_s + akg_s + phe__L_s
0.0007093732888843219
100.0 %


gly_s
gly_c + h_c --> -4.27350427350427e-6*sigma PORE-CPLX_s + gly_s + h_s
0.0014233087303357351
94.76281121891273 %


leu__L_s
h_c + leu__L_c --> -4.27350427350427e-6*sigma PORE-CPLX_s + h_s + leu__L_s
0.003095526244994408

In [11]:
pd.DataFrame.from_dict(dct).T.to_csv("./Supplementary Files/SupplementaryFile_AA_Production.csv")